###**G345 - ANALISIS DE DATOS**

##**Análisis descriptivo de la contribución energética, económica y ambiental de los proyectos FNCER en la matriz energética colombiana**

Nombres y cédula de los integrantes del equipo

*   Karen Alejandra Jaramillo López 1128905063
*   Juan Esteban Albornoz Gil 1089601133
*   Óscar Andrés García Jurado      1002545539
*   Juan Pablo Gonzalez Vásquez     1004701517


##**Preparar el entorno**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Se importan las librerías
import numpy as np
import pandas as pd
import os

#import matplotlib.pyplot as plt
#import seaborn as sns
#import statsmodels.api as sm

##**Cargar DataFrame desde un xlsx**

###**Acerca de este archivo**

Comprende los reportes de la capacidad instalada a partir de FNCER instaladas o en pruebas en la matriz de generación. Se tienen en cuenta en cada corte los proyectos reportados directamente por los agentes, proyectos registrados en PARATEC (http://paratec.xm.com.co/paratec/SitePages/generacion.aspx?q=capacidad ) y pequeños proyectos reportados por los OR. Las metas son medidas en distintos horizontes. Para finalizar el gobierno el Plan Nacional de Desarrollo estableció una meta de 1500 MW. No obstante, a finales de 2022 se pueden superar los 2000 MW.

###**Diccionario de Datos: Esquema del xlsx**

**Proyecto:** Identificador de cada uno de los proyectos que hacen parte de las metas en cada corte reportado.

**Tipo:** Tecnología FNCER correspondiente el proyecto, Ejm: Solar y Eólica.

**Capacidad:** Capacidad instalada en MW del proyecto relacionado. Corresponde a la Capacidad Efectiva Neta, no a la Capacidad Pico.

**Departamento:** Nombre del departamento del proyecto referenciado.

**Municipio:** Nombre del municipio del proyecto referenciado, para proyectos grandes se toma el municipio principal.

**Código Departamento:** Código del Departamento de acuerdo a DIVIPOLA.

**Código Municipio:** Código del Municipio de acuerdo a DIVIPOLA.

**Fecha estimada FPO:** Fecha en la que fueron reportados los nuevos proyectos, puede ser la Fecha de Puesta en Operación - FPO oficial o la fecha reportada en la que arrancaron pruebas.

**Energía [kWh/día]:** Energía estimada que produce la central en kWh/día, corresponde a un factor de planta de 0,2 para energía solar y 0,4 para las demás tecnologías.

**Usuarios:** Equivalente de los usuarios residenciales que puede atender el proyecto de generación tomando como referencia el factor de planta y un consumo de subsistencia por usuario de 173 kWh/mes.

**Inversión estimada [COP]:** Inversión en pesos estimada indirectamente con la potencia de cada planta.

**Empleos estimados:**
Número aproximado de empleos directos calculados de acuerdo a referencias bibliográficas en función de la capacidad de la central.

**Emisiones CO2 [ton/año]:** Cantidad de emisiones de CO2 estimadas tomando como referencia la energía de la central en un año y el factor de emisiones de 0,8 gr/kWh correspondiente a una central a carbón.

###**Archivo xlxs**

In [2]:
# Lectura del archivo .xlsx

try:
  df = pd.read_excel('Bases de datos.xlsx', sheet_name='FNCER_20260117')

  # Mostramos las 5 primeras filas del DataFrame y sus dimensiones
  print(df.shape)

except:
  print("No se pudo abrir el archivo.")

(164, 12)


##**Limpieza y Preparación de Datos**

####**Exploración inicial del DataFrame**

In [ ]:
df.head()

,Proyecto,Tipo,Capacidad,Departamento,Municipio,Código Departamento,Código Municipio,Fecha estimada FPO,Energía [kWh/día],Usuarios,Inversión estimada [COP],Empleos estimados,Emisiones CO2 [Ton/año]
0,JEPIRACHI,Eólico,18.42,LA GUAJIRA,URIBIA,44,44847,2004-04-27 00:00:00,176832,30665,79206000000,51,51635
1,AUTOG CELSIA SOLAR YUMBO,Solar,9.80,VALLE DEL CAUCA,YUMBO,76,76892,2017-03-09 00:00:00,47040,8157,42140000000,68,13736
2,CELSIA SOLAR BOLIVAR,Solar,8.06,BOLÍVAR,SANTA ROSA,13,13683,2019-02-01 00:00:00,38688,6709,34658000000,56,11297
3,EL PASO SOLAR (ENEL GREEN POWER),Solar,67.00,CESAR,EL PASO,20,20250,2019-04-05 00:00:00,321600,55769,288100000000,466,93907
4,SOLAR CASTILLA ECP,Solar,20.00,META,CASTILLA LA NUEVA,50,50150,2019-10-10 00:00:00,96000,16647,86000000000,139,28032


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Proyecto                  158 non-null    object 
 1   Tipo                      158 non-null    object 
 2   Capacidad                 158 non-null    float64
 3   Departamento              158 non-null    object 
 4   Municipio                 158 non-null    object 
 5   Código Departamento       158 non-null    int64  
 6   Código Municipio          158 non-null    int64  
 7   Fecha estimada FPO        158 non-null    object 
 8   Energía [kWh/día]         158 non-null    int64  
 9   Usuarios                  158 non-null    int64  
 10  Inversión estimada [COP]  158 non-null    int64  
 11  Empleos estimados         158 non-null    int64  
 12  Emisiones CO2 [Ton/año]   158 non-null    int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 16.2+ KB


In [ ]:
for var in df.columns:
  if df[var].nunique() < 50:
    print(f"{var}[n = {df[var].nunique()}]: {df[var].unique()}")
  else:
    print(f"{var}[n = {df[var].nunique()}]")

Proyecto[n = 156]
Tipo[n = 2]: ['Eólico' 'Solar']
Capacidad[n = 104]
Departamento[n = 26]: ['LA GUAJIRA' 'VALLE DEL CAUCA' 'BOLÍVAR' 'CESAR' 'META'
 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA' 'ATLÁNTICO'
 'TOLIMA' 'ANTIOQUIA' 'CHOCÓ' 'RISARALDA' 'CÓRDOBA' 'GUAINÍA' 'SANTANDER'
 'CALDAS' 'HUILA' 'ARAUCA' 'BOYACÁ' 'BOGOTÁ D.C.' 'NORTE DE SANTANDER'
 'CASANARE' 'SUCRE' 'CAUCA' 'QUINDÍO' 'MAGDALENA' 'CUNDINAMARCA']
Municipio[n = 79]
Código Departamento[n = 26]: [44 76 13 20 50 88  8 73  5 27 66 23 94 68 17 41 81 15 11 54 85 70 19 63
 47 25]
Código Municipio[n = 81]
Fecha estimada FPO[n = 70]
Energía [kWh/día][n = 106]
Usuarios[n = 106]
Inversión estimada [COP][n = 112]
Empleos estimados[n = 75]
Emisiones CO2 [Ton/año][n = 106]


In [ ]:
df.isnull().sum()

,0
Proyecto,0
Tipo,0
Capacidad,0
Departamento,0
Municipio,0
Código Departamento,0
Código Municipio,0
Fecha estimada FPO,0
Energía [kWh/día],0
Usuarios,0


####**Inicio de modificaciones en el DataFrame**

In [ ]:
# Encontrar los valores erróneos en el df pertenecientes a Municipio identificado con ""
filtro = df['Municipio'].str.contains('""')
print(f"Cantidad de registros erróneos: {df['Municipio'][filtro].count()}")
print(f"Dimensión: {df.shape}")

Cantidad de registros erróneos: 5
Dimensión: (158, 13)


In [ ]:
# Identificamos los índices pertenecientes a los valores erróneos y aplicamos la máscara
indices_erroneos = df.index[filtro]

print("Índices con valores erróneos:")
print(indices_erroneos)

Índices con valores erróneos:
Index([134, 147, 152, 153, 154], dtype='int64')


In [ ]:
# Se crea diccionario que almacenará las correcciones de los valores erróneos y se reemplazan en df usando .loc
correcciones = {
    134: "TIERRALTA",
    147: "LA ESPERANZA",
    152: "MONTELÍBANO",
    153: "BUGA",
    154: "BUGA"
}

for idx, municipio in correcciones.items():
    df.loc[idx, "Municipio"] = municipio

# Verificamos nuevamente existencia de datos erróneos
restantes = df['Municipio'].str.contains('""', na=False)

print("Registros erróneos restantes:", restantes.sum())

Registros erróneos restantes: 0


In [ ]:
df.loc[134]

,134
Proyecto,AQUASOL
Tipo,Solar
Capacidad,1.5
Departamento,CÓRDOBA
Municipio,TIERRALTA
Código Departamento,23
Código Municipio,23
Fecha estimada FPO,2022-10-31 00:00:00
Energía [kWh/día],7200
Usuarios,1249


In [ ]:
df.loc[147]


,147
Proyecto,Parque Fotovoltaico Sunnorte
Tipo,Solar
Capacidad,35.0
Departamento,NORTE DE SANTANDER
Municipio,LA ESPERANZA
Código Departamento,54
Código Municipio,54
Fecha estimada FPO,2023-12-31 00:00:00
Energía [kWh/día],168000
Usuarios,29133


In [ ]:
df.loc[152]

,152
Proyecto,Parque Fotovoltaico Montelíbano Solar
Tipo,Solar
Capacidad,9.9
Departamento,CÓRDOBA
Municipio,MONTELÍBANO
Código Departamento,23
Código Municipio,23
Fecha estimada FPO,2022-10-30 00:00:00
Energía [kWh/día],47520
Usuarios,8240


In [ ]:
df.loc[153]

,153
Proyecto,BUGA 1 PARTE 2
Tipo,Solar
Capacidad,5.94
Departamento,VALLE DEL CAUCA
Municipio,BUGA
Código Departamento,76
Código Municipio,76
Fecha estimada FPO,2022-12-31 00:00:00
Energía [kWh/día],28512
Usuarios,4944


In [ ]:
df.loc[154]

,154
Proyecto,BUGA 1 PARTE 1
Tipo,Solar
Capacidad,3.96
Departamento,VALLE DEL CAUCA
Municipio,BUGA
Código Departamento,76
Código Municipio,76
Fecha estimada FPO,2022-07-31 00:00:00
Energía [kWh/día],19008
Usuarios,3296


In [ ]:
df.tail()

,Proyecto,Tipo,Capacidad,Departamento,Municipio,Código Departamento,Código Municipio,Fecha estimada FPO,Energía [kWh/día],Usuarios,Inversión estimada [COP],Empleos estimados,Emisiones CO2 [Ton/año]
153,BUGA 1 PARTE 2,Solar,5.94,VALLE DEL CAUCA,BUGA,76,76,2022-12-31 00:00:00,28512,4944,22275000000,41,8326
154,BUGA 1 PARTE 1,Solar,3.96,VALLE DEL CAUCA,BUGA,76,76,2022-07-31 00:00:00,19008,3296,14850000000,28,5550
155,REFINERÍA GRB,Solar,21.00,SANTANDER,BARRANCABERMEJA,68,68081,2023-12-31 00:00:00,100800,17480,78750000000,146,29434
156,EMSA GD Y AG (34 Proyectos),Solar,1.90,META,VILLAVICENCIO,50,50001,2022-05-31 00:00:00,9120,1582,8170000000,13,2663
157,ENELAR GD Y AG (4 Proyectos),Solar,0.11,ARAUCA,ARAUCA,81,81001,2022-05-31 00:00:00,528,92,473000000,1,154


In [ ]:
# Pasar la fecha de tipo object a tipo datetime
df['Fecha estimada FPO'] = pd.to_datetime(
    df['Fecha estimada FPO'],
    format='%m/%d/%y %H:%M',
    errors='coerce'
)
df['Fecha estimada FPO'].dtype

dtype('<M8[ns]')

In [ ]:
print(df['Fecha estimada FPO'].dtype.name)

datetime64[ns]


In [ ]:
df[df['Fecha estimada FPO'].isna()]

,Proyecto,Tipo,Capacidad,Departamento,Municipio,Código Departamento,Código Municipio,Fecha estimada FPO,Energía [kWh/día],Usuarios,Inversión estimada [COP],Empleos estimados,Emisiones CO2 [Ton/año]
87,OCELOTE,Solar,14.0,META,PUERTO GAITÁN,50,50568,NaT,67200,11653,52500000000,97,19622
109,PUERTA DE ORO Fase 1,Solar,120.0,CUNDINAMARCA,GUADUAS,25,25320,NaT,576000,99884,450000000000,835,168192
142,PUERTA DE ORO Fase 2,Solar,180.0,CUNDINAMARCA,GUADUAS,25,25320,NaT,864000,149827,675000000000,1253,252288


In [ ]:
df['Fecha estimada FPO'].min()

Timestamp('2004-04-27 00:00:00')

In [ ]:
df['Fecha estimada FPO'].max()

Timestamp('2024-06-30 00:00:00')